In [ ]:
import numpy as np
import pyrender
import trimesh
from utils.spherical_harmonics import *
import scipy
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import griddata
from scipy.integrate import dblquad
import numpy as np
from scipy.optimize import minimize
from scipy.special import sph_harm
from scipy.spatial.transform import Rotation
import os

In [ ]:
def real_spherical_harmonic(m, l, phi, theta):
    assert np.abs(m) <= l
    if m == 0:
        return np.real(scipy.special.sph_harm(m, l, phi, theta))
    elif m > 0:
        return np.real((1/np.sqrt(2))*(scipy.special.sph_harm(m, l, phi, theta) + ((-1)**m) * scipy.special.sph_harm(-m, l, phi, theta)))
    elif m < 0:
        return np.real((1/(1j*np.sqrt(2)))*(scipy.special.sph_harm(-m, l, phi, theta) - ((-1)**(-m)) * scipy.special.sph_harm(m, l, phi, theta)))

In [ ]:
def simpson_integral_grids(x_grid, y_grid, eval_grid):
    a = x_grid[0,0]
    b = x_grid[0,-1]
    n = int(x_grid.shape[0]/2)
    
    c = y_grid[0,0]
    d = y_grid[-1,0]
    m = int(y_grid.shape[1]/2)

    h = (b - a) / (2*n)
    J1 = 0  # Terminos extremos
    J2 = 0  # Terminos pares
    J3 = 0  # Terminos impares
    for i in range(2*n + 1):  # i in [0,2n]
        hx = (d - c) / (2*m)
        K1 = eval_grid[i, 0] + eval_grid[i, -1]  # Terminos extremos
        K2 = 0  # Terminos pares
        K3 = 0  # Terminos impares
        for j in range(1, 2*m):  # j in [1,2m-1]
            Q = eval_grid[i, j]
            if j % 2 == 0:
                K2 += Q
            else:
                K3 += Q
        L = (K1 + 2*K2 + 4*K3)*hx/3
        if i == 0 or i == 2*n:
            J1 += L
        elif i % 2 == 0:
            J2 += L
        else:
            J3 += L
    J = h*(J1 + 2*J2 + 4*J3)/3
    return J

In [ ]:
def simpson_integral_call(func, a, b, c, d, n, m):
    h = (b - a) / (2*n)
    J1 = 0  # Terminos extremos
    J2 = 0  # Terminos pares
    J3 = 0  # Terminos impares
    for i in range(2*n + 1):  # i in [0,2n]
        x = a + i*h
        hx = (d - c) / (2*m)
        K1 = func(x, c) + func(x, d)
        K2 = 0
        K3 = 0
        for j in range(1, 2*m):  # j in [1,2m-1]
            y = c + j*hx
            Q = func(x, y)
            if j % 2 == 0:
                K2 += Q
            else:
                K3 += Q
        L = (K1 + 2*K2 + 4*K3)*hx/3
        if i == 0 or i == 2*n:
            J1 += L
        elif i % 2 == 0:
            J2 += L
        else:
            J3 += L
    J = h*(J1 + 2*J2 + 4*J3)/3
    return J

In [ ]:
def print_coefficients(coefficients):
    print("Coeficients (m,l)")
    l_max = int(np.sqrt(len(coefficients)))
    cont = 0
    longitud_maxima_coeficientes = max(len("{:.2f}".format(x)) for x in coefficients)
    longitud_maxima_valores = max(len(f"({m},{l})") for l in range(0, max_l) for m in range(-l, l + 1))
    for l in range(0, max_l + 1):
        for m in range(-l, l + 1):
            print("{:>{}}:".format(f"({m},{l})", longitud_maxima_valores), end=" ")
            print("{:>{}}".format("{:.2f}".format(coefficients[cont]), longitud_maxima_coeficientes), end=", ")
            cont += 1
        print("")

In [ ]:
def compute_grids(r, phi, theta, shape = (201,201)):
    n_phi, n_theta = shape
    p = np.linspace(0, 2*np.pi, n_phi)
    t = np.linspace(0, np.pi, n_theta)

    phi_grid, theta_grid = np.meshgrid(p, t) # phi over cols, theta over rows
    # griddata((points), values, (Points at which to interpolate data))
    r_grid = griddata((phi, theta), r, (phi_grid, theta_grid), method='nearest', fill_value=0.0)
    return r_grid, phi_grid, theta_grid

def coefficient(m, l, phi_grid, theta_grid, r_grid):
    eval_grid = r_grid * np.sin(theta_grid) * real_spherical_harmonic(m, l, phi_grid, theta_grid)
    return simpson_integral_grids(phi_grid, theta_grid, eval_grid)

def spherical_coefficients(max_l, phi_grid, theta_grid, r_grid):
    coefficients = []
    for l in range(0, max_l + 1):
        for m in range(-l, l + 1):
            coefficients.append(coefficient(m, l, phi_grid, theta_grid, r_grid))
    return coefficients

In [ ]:

mesh = trimesh.load('../data/oriented_dataset/Dundee 2ROW British/DHBT16P16 4.stl')
x = mesh.vertices[:, 0]
y = mesh.vertices[:, 1]
z = mesh.vertices[:, 2]
r, phi, theta = spherical_coordinates(x, y, z)

r_grid, phi_grid, theta_grid = compute_grids(r, phi, theta, shape = (201,201))

max_l = 10
coefs = spherical_coefficients(max_l, phi_grid, theta_grid, r_grid)

print_coefficients(coefs)

In [ ]:
mesh = trimesh.load('../data/oriented_dataset/Dundee 6ROW Scandinavian/DHBT16P101 5.stl')
x = mesh.vertices[:, 0]
y = mesh.vertices[:, 1]
z = mesh.vertices[:, 2]
r, phi, theta = spherical_coordinates(x, y, z)
r_grid, phi_grid, theta_grid = compute_grids(r, phi, theta, shape = (201,201))

max_l = 30

for current_l in range(30, 52, 3):
    coefs = spherical_coefficients(current_l, phi_grid, theta_grid, r_grid)
    r_new = np.zeros(r.shape)
    cont = 0
    for l in range(0, current_l + 1):
        for m in range(-l, l + 1):
            r_new += coefs[cont] * real_spherical_harmonic(m, l, phi, theta)
            cont += 1

    x, y, z = cartesian_coordinates_3d(r_new, phi, theta)
    verts = np.column_stack((x,y,z))
    mesh.vertices = verts

    export_path = "../data/shperical_plotting/Dundee 6ROW Scandinavian DHBT16P101 5/"
    os.makedirs(export_path, exist_ok=True)
    mesh.export(export_path + f'L{current_l}.stl')

In [ ]:
mesh = trimesh.load('../data/oriented_dataset/Dundee 6ROW Scandinavian/DHBT16P101 5.stl')
export_path = "../data/shperical_plotting/6ROW/"

mesh.export(export_path + f'DHBT16P101 5.stl')

In [1]:
from utils.spherical_harmonics import *

In [2]:
dataset_path = "../data/Barley STL oriented files"
exportt_path = "../data/spherical_coefficients_L30"
spherical_harmonics_coefficients_dataset(dataset_path, exportt_path, max_l = 30, shape = (201, 201))

Entrando en la carpeta ../data/Barley STL oriented files
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 2ROW British
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 2ROW Scottish
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 6ROW BERE Orkney
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 6ROW BERE Unknown
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 6ROW BERE Western Isles
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 6ROW Faro
Entrando en la carpeta ../data/Barley STL oriented files/Dundee 6ROW Scandinavian
Entrando en la carpeta ../data/Barley STL oriented files/Orkney 2ROW British
Entrando en la carpeta ../data/Barley STL oriented files/Orkney 2ROW Scottish
Entrando en la carpeta ../data/Barley STL oriented files/Orkney 6ROW BERE ORKNEY
Entrando en la carpeta ../data/Barley STL oriented files/Orkney 6ROW BERE Unknown
Entrando en la carpeta ../data/Barley STL oriented files/Orkney